In [5]:
import rdflib
from rdflib import Namespace, URIRef, Literal, Graph
from rdflib.namespace import RDF, OWL, DCTERMS, FOAF, SKOS
import pandas as pd
import re

root = "https://w3id.org/the-elephant-man/"
SCHEMA = Namespace("https://schema.org/")
MO = Namespace("http://musicontology.com/")
DBO = Namespace("https://dbpedia.org/ontology/")
VIR = Namespace("http://w3id.org/vir")
FRBR = Namespace("http://purl.org/vocab/frbr/core")
SWC = Namespace("http://data.semanticweb.org/ns/swc/ontology ")
FABIO = Namespace("http://purl.org/spar/fabio")
CRM = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
GNDO = Namespace("http://d-nb.info/standards/elementset/gnd")
MADS = Namespace("http://www.loc.gov/mads/rdf/v1#")
GVP = Namespace("http://vocab.getty.edu/ontology")
FRAPO = Namespace("http://purl.org/cerif/frapo")

namespace_mappings = {
    "SCHEMA": Namespace("https://schema.org/"),
    "MO": Namespace("http://musicontology.com/"),
    "DBO": Namespace("https://dbpedia.org/ontology/"),
    "VIR": Namespace("http://w3id.org/vir"),
    "FRBR": Namespace("http://purl.org/vocab/frbr/core"),
    "SWC": Namespace("http://data.semanticweb.org/ns/swc/ontology "),
    "FABIO": Namespace("http://purl.org/spar/fabio/"),
    "CRM": Namespace("http://www.cidoc-crm.org/cidoc-crm/"),
    "GNDO": Namespace("http://d-nb.info/standards/elementset/gnd"),
    "MADS": Namespace("http://www.loc.gov/mads/rdf/v1#"),
    "GVP": Namespace("http://vocab.getty.edu/ontology"),
    "FRAPO": Namespace("http://purl.org/cerif/frapo"),
    "WD": Namespace("http://www.wikidata.org/entity"),
    "RDF": RDF,
    "OWL": OWL,
    "DCTERMS": DCTERMS,
    "FOAF": FOAF,
    "SKOS": SKOS
}


# Function to resolve prefixed properties to full URIRefs
def resolve_prefixed_property(prefixed_property):
    namespace, local_name = prefixed_property.split(":")
    # crm case
    if namespace == 'crm':
        local_name = local_name.split(' ')[0]
    namespace = namespace_mappings[namespace.upper()]

    return namespace[local_name]


def matches_pattern(input_string):
    pattern = r"\w+:\w+"
    return bool(re.match(pattern, input_string))


# Access csv
csv_files_list = ["JournalArticle.csv"]
csv_folder_path = "../items-csv/"
internal_uri_types = [
    "person/",
    "item/",
    "place/",
    "time/",
    "concept/",
    "language",
    "organization",
    "occupation",
]

g = Graph()
for csv_file in csv_files_list:
    df = pd.read_csv(csv_folder_path + csv_file, delimiter=";")
    for _, row in df.iterrows():
        # subject
        s = URIRef(root + row.iloc[0].replace(' ', '-'))
        print('subjects')
        print(s)
        # predicate
        p = resolve_prefixed_property(row.iloc[1])
        print('predicate')
        print(p)
        # object
        o = row.iloc[2]
        # check if object contains part of the internal uri
        if any(uri_type in o for uri_type in internal_uri_types):
            o = URIRef(root + o.replace(' ', '-'))
        elif matches_pattern(o):
            o = resolve_prefixed_property(o)
        elif 'crm:' in o: 
            o = namespace_mappings['CRM'] + o.split(':')[1].split('_')[0]
        else: 
            o = Literal(o)
        print('object')
        print(o)

subjects
https://w3id.org/the-elephant-man/item/Journal_Article
predicate
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
object
http://purl.org/spar/fabio/JournalArticle
subjects
https://w3id.org/the-elephant-man/item/Journal_Article
predicate
http://purl.org/dc/terms/subject
object
Proteus syndrome
subjects
https://w3id.org/the-elephant-man/item/Journal_Article
predicate
http://purl.org/dc/terms/subject
object
https://w3id.org/the-elephant-man/person/Joseph_Merrick
subjects
https://w3id.org/the-elephant-man/item/Journal_Article
predicate
https://schema.org/identifier
object
10.1136/bmj.293.6548.683
subjects
https://w3id.org/the-elephant-man/item/Journal_Article
predicate
https://schema.org/datePublished
object
1 September 1986
subjects
https://w3id.org/the-elephant-man/item/Journal_Article
predicate
http://purl.org/dc/terms/title
object
The Proteus syndrome: the Elephant Man diagnosed
subjects
https://w3id.org/the-elephant-man/item/Journal_Article
predicate
https://schema.org/genre
o